# Introduction

During the fine tuning process dataset and format preparation playing a key role.

Important Note - Use alpaca or chatml format for base model finetuning and other formats like llama, mistral, phi in "INSTRUCT" model. so, basic concept is if your model is trained or instruct on certain formatting then we should utilize the same format when feeding the data otherwise we will confuse the model. 


**Different Type of formats**

1) Aplaca Format

This format is defined by standford unity which is used in **training base models**

**Format** - 
    Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

    ### Instruction:
    {instruction}

    ### Input:
    {input}

    ### Response:

**Use Case** - 
    To train a base model of llama , mistral or Qwen you can utilize this format to train it. 

**Reference** - 
    https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-Alpaca.ipynb

    Where we are converting a dataset to alpaca format before processing, below is the code snippet for it


In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

2. ChatML format

Chat ML format is again same as alpaca model is used for training a base model. And this is like chat gpt format where, it takes only input and output but no instruction part, so basically we combine Input = Instruction + Input part for the representation. 



Format - 


```python
<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
What's the capital of France?<|im_end|>
<|im_start|>assistant
Paris.
```



3. LLAMA Chat format

LLAMA chat format or any other model specific chat format are used on "INSTRUCT" model 


Format - 

```python
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Hello!<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hey there! How are you?<|eot_id|><|start_header_id|>user<|end_header_id|>

I'm great thanks!<|eot_id|>
```



# Data formatting that you will see commonly used


Data Format 1 - 

so basically, any text dataset, we can convert in this format and then eventually apply "apply_chat_template" based on tokenizer is used. 

To give you an example, let's say if we are using LLAMA3.1 model, and we have the data in below format, we can utilize LLAMA3.1 tokenizer and apply_chat_template which will apply instruct model training format and create a text field out of it example you will see at the end


```python
{'conversations': [{'content': 'What are the three primary colors?',
   'role': 'user'},
  {'content': 'The three primary colors are red, blue, and yellow. These colors are called primary because they cannot be created by mixing other colors and all other colors can be made by combining them in various proportions. In the additive color system, used for light, the primary colors are red, green, and blue (RGB).',
   'role': 'assistant'},
  {'content': 'Identify 3 different features of the given product.\nYour input is:\nApple Watch Series 6',
   'role': 'user'},
  {'content': "Three features of the Apple Watch Series 6 are:\n\n1. Blood Oxygen Monitoring: The Apple Watch Series 6 includes a Blood Oxygen sensor that measures the oxygen saturation (SpO2) of the wearer’s blood. It uses infrared and red LEDs to take measurements, alongside photodiodes that measure the light reflected from the blood in the user's veins.\n\n2. Always-On Retina Display: The Always-On Retina display ensures that the watch face is always visible, even when the user’s wrist is down. This makes it easier to check the time, notifications, and other important information without having to raise your wrist or tap on the screen.\n\n3. Fitness Tracking: The Apple Watch Series 6 has a range of features designed to help users track their fitness and stay active. These include tracking workouts, monitoring daily activity levels, and setting reminders to stand or move around periodically. The watch also includes a heart rate monitor to track cardio fitness levels during exercise.",
   'role': 'assistant'},
  {'content': "Given a description of a character, come up with possible motivations for the character's behaviour.\nYour input is:\nThe character is a young man who is often confrontational and rebellious.",
   'role': 'user'},
  {'content': '1. The young man may have grown up in a challenging home environment where he had to fight to be heard, leading him to develop confrontational behaviors.\n\n2. He may be struggling with feelings of insecurity or low self-worth, leading him to act out in rebellious ways to gain attention or assert his individuality.\n\n3. The young man may have experienced a traumatic event or loss, causing him to feel anger and resentment towards the world, leading him to behave confrontationally.\n\n4. He may feel misunderstood or stifled by societal expectations, leading him to rebel against norms and conventions as a means of expressing his frustration.\n\n5. The young man may be searching for a sense of identity or purpose, causing him to engage in confrontational behavior as a way of defining and asserting himself.\n\n6. He may be influenced by a group of peers who engage in rebellious behavior, causing him to adopt similar behaviors in order to fit in with the group.',
   'role': 'assistant'}]}
```

AFTER applying chat format of llama3.1

```python
{'conversations': [{'content': 'What are the three primary colors?',
   'role': 'user'},
  {'content': 'The three primary colors are red, blue, and yellow. These colors are called primary because they cannot be created by mixing other colors and all other colors can be made by combining them in various proportions. In the additive color system, used for light, the primary colors are red, green, and blue (RGB).',
   'role': 'assistant'},
  {'content': 'Identify 3 different features of the given product.\nYour input is:\nApple Watch Series 6',
   'role': 'user'},
  {'content': "Three features of the Apple Watch Series 6 are:\n\n1. Blood Oxygen Monitoring: The Apple Watch Series 6 includes a Blood Oxygen sensor that measures the oxygen saturation (SpO2) of the wearer’s blood. It uses infrared and red LEDs to take measurements, alongside photodiodes that measure the light reflected from the blood in the user's veins.\n\n2. Always-On Retina Display: The Always-On Retina display ensures that the watch face is always visible, even when the user’s wrist is down. This makes it easier to check the time, notifications, and other important information without having to raise your wrist or tap on the screen.\n\n3. Fitness Tracking: The Apple Watch Series 6 has a range of features designed to help users track their fitness and stay active. These include tracking workouts, monitoring daily activity levels, and setting reminders to stand or move around periodically. The watch also includes a heart rate monitor to track cardio fitness levels during exercise.",
   'role': 'assistant'},
  {'content': "Given a description of a character, come up with possible motivations for the character's behaviour.\nYour input is:\nThe character is a young man who is often confrontational and rebellious.",
   'role': 'user'},
  {'content': '1. The young man may have grown up in a challenging home environment where he had to fight to be heard, leading him to develop confrontational behaviors.\n\n2. He may be struggling with feelings of insecurity or low self-worth, leading him to act out in rebellious ways to gain attention or assert his individuality.\n\n3. The young man may have experienced a traumatic event or loss, causing him to feel anger and resentment towards the world, leading him to behave confrontationally.\n\n4. He may feel misunderstood or stifled by societal expectations, leading him to rebel against norms and conventions as a means of expressing his frustration.\n\n5. The young man may be searching for a sense of identity or purpose, causing him to engage in confrontational behavior as a way of defining and asserting himself.\n\n6. He may be influenced by a group of peers who engage in rebellious behavior, causing him to adopt similar behaviors in order to fit in with the group.',
   'role': 'assistant'}],
 'text': "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nBelow are some instructions that describe some tasks. Write responses that appropriately complete each request.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhat are the three primary colors?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThe three primary colors are red, blue, and yellow. These colors are called primary because they cannot be created by mixing other colors and all other colors can be made by combining them in various proportions. In the additive color system, used for light, the primary colors are red, green, and blue (RGB).<|eot_id|><|end_of_text|><|start_header_id|>user<|end_header_id|>\n\nIdentify 3 different features of the given product.\nYour input is:\nApple Watch Series 6<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThree features of the Apple Watch Series 6 are:\n\n1. Blood Oxygen Monitoring: The Apple Watch Series 6 includes a Blood Oxygen sensor that measures the oxygen saturation (SpO2) of the wearer’s blood. It uses infrared and red LEDs to take measurements, alongside photodiodes that measure the light reflected from the blood in the user's veins.\n\n2. Always-On Retina Display: The Always-On Retina display ensures that the watch face is always visible, even when the user’s wrist is down. This makes it easier to check the time, notifications, and other important information without having to raise your wrist or tap on the screen.\n\n3. Fitness Tracking: The Apple Watch Series 6 has a range of features designed to help users track their fitness and stay active. These include tracking workouts, monitoring daily activity levels, and setting reminders to stand or move around periodically. The watch also includes a heart rate monitor to track cardio fitness levels during exercise.<|eot_id|><|end_of_text|><|start_header_id|>user<|end_header_id|>\n\nGiven a description of a character, come up with possible motivations for the character's behaviour.\nYour input is:\nThe character is a young man who is often confrontational and rebellious.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n1. The young man may have grown up in a challenging home environment where he had to fight to be heard, leading him to develop confrontational behaviors.\n\n2. He may be struggling with feelings of insecurity or low self-worth, leading him to act out in rebellious ways to gain attention or assert his individuality.\n\n3. The young man may have experienced a traumatic event or loss, causing him to feel anger and resentment towards the world, leading him to behave confrontationally.\n\n4. He may feel misunderstood or stifled by societal expectations, leading him to rebel against norms and conventions as a means of expressing his frustration.\n\n5. The young man may be searching for a sense of identity or purpose, causing him to engage in confrontational behavior as a way of defining and asserting himself.\n\n6. He may be influenced by a group of peers who engage in rebellious behavior, causing him to adopt similar behaviors in order to fit in with the group.<|eot_id|><|end_of_text|>"}
```



# Unsloth or huggingface for applying the chat format to data to create a text field

1) Unsloth

Unsloth apply chat format, so basically unsloth provides a utility method where you can supply any template that you want to use, and pass your conversation into the method and it will apply appropriate formatting on it 


```python
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
    map_eos_token = True, # Maps <|im_end|> to </s> instead
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("philschmid/guanaco-sharegpt-style", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)
```


2) HuggingFace - 

if you actually create your dataset in below format than actually you can pass directly to SFTTrainer class, and SFTTrainer class eventually use tokenizer and apply appropriate formatting for you so basically you don't need and processing on your end. 

```python
{"messages": [{"role": "system", "content": "You are helpful"}, {"role": "user", "content": "What's the capital of France?"}, {"role": "assistant", "content": "..."}]}
{"messages": [{"role": "system", "content": "You are helpful"}, {"role": "user", "content": "Who wrote 'Romeo and Juliet'?"}, {"role": "assistant", "content": "..."}]}
{"messages": [{"role": "system", "content": "You are helpful"}, {"role": "user", "content": "How far is the Moon from Earth?"}, {"role": "assistant", "content": "..."}]}
```

Reference - https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb 

Check this vision model reference where we are passing the data in following format and rest is take care of 

```python
{'messages': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'You are an expert radiographer. Describe accurately what you see in this image.'},
    {'type': 'image',
     'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=657x442>}]},
  {'role': 'assistant',
   'content': [{'type': 'text',
     'text': 'Panoramic radiography shows an osteolytic lesion in the right posterior maxilla with resorption of the floor of the maxillary sinus (arrows).'}]}]}
```
